In [ ]:
!pip install pytorch-lightning
!pip install transformers

!pip install wandb
!pip install bitsandbytes-cuda113

In [ ]:
!pip install x-transformers==0.25.2

In [1]:
!gdown https://drive.google.com/uc?id=1-3DvN7vCzagRmVpITSjbT7Z6qpaiUS4x

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Downloading...
From: https://drive.google.com/uc?id=1-3DvN7vCzagRmVpITSjbT7Z6qpaiUS4x
To: /home/alex/experements_/palm/wiki_prepared.txt
100%|██████████████████████████████████████| 7.48G/7.48G [05:51<00:00, 21.3MB/s]


In [2]:
!wget -nc https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
!tar -xzvf gazeta_jsonl.tar.gz

File ‘gazeta_jsonl.tar.gz’ already there; not retrieving.

gazeta_train.jsonl
gazeta_test.jsonl
gazeta_val.jsonl


In [2]:
a = [1,2,2]
from tqdm import tqdm

for i in tqdm(a):
    print(i)
    

100%|██████████| 3/3 [00:00<00:00, 21435.97it/s]

1
2
2


In [ ]:
!wandb login



wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#

In [ ]:
import os

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import  GPT2TokenizerFast

BASE_MODEL = "sberbank-ai/rugpt3small_based_on_gpt2"

tokenizer = GPT2TokenizerFast.from_pretrained(BASE_MODEL, bos_token='<|endoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')



Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
class SumDataset(Dataset):
    def __init__(self):
      self.max_length = 1024
      self.prepared = []
      df = pd.read_json('gazeta_train.jsonl',lines=True)
      
      for sum, text in zip(df['summary'],df['text']):
        pr = f'<s>{text} короче говоря {sum}'
        
        enc = self._encode(text=pr)
        if len(enc)<1024:
          self.prepared+=[enc]
      

    def __len__(self):
        return len(self.prepared)

    def __getitem__(self, item):
        return torch.tensor(self.prepared[item], dtype=torch.long)

    def _encode(self, text):
        
        encoded_sample = tokenizer.encode(text, truncation=True, max_length=self.max_length, padding="max_length")

        return encoded_sample

In [ ]:
from tqdm import tqdm

class Dataset_processing(Dataset):
    def __init__(self):
        super().__init__()

        

        self.exmpls = []
        self.eos = tokenizer.eos_token
        
        
        
        
        with open('wiki_prepared.txt') as f:
          lines = f.readlines()
        data = []
        for mes in lines[:100000]:
            data+=[mes]
        
        descriptions = data
        
        
        self.max_length = 1024
        
        for text in tqdm(descriptions,desc='Tokenizing'):
          txt = text + self.eos

          try:
            encoded = tokenizer.encode(txt,  truncation=True, max_length=self.max_length, padding="max_length")
            self.exmpls.append(encoded)
          except Exception as e:
            print(e)
          

          
        
        
    def __len__(self):
        return len(self.exmpls)

    def __getitem__(self, idx):
        return torch.tensor(self.exmpls[idx], dtype=torch.long)

In [ ]:
#data_wiki = Dataset_processing()

In [ ]:
#train_dev_sets = torch.utils.data.ConcatDataset([s, data_wiki])

In [ ]:
class Args():
  def __init__(self):
    self.train_batch_size = 4
    self.lr = 1e-4
    self.n_training_steps = 10

args = Args()

#args.n_training_steps = 100000//args.train_batch_size
#args.n_warmup_steps = args.n_training_steps//5

In [ ]:
#from sklearn.model_selection import train_test_split
#train_dataset, val_dataset = train_test_split(data_wiki, test_size=100, random_state=42, shuffle=True)

In [ ]:
#tokenizer.encode_plus('how to poop')

{'input_ids': [76, 1170, 710, 12933, 1661], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
#args.n_training_steps 

10000

In [ ]:
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
class DecoderDataModule(pl.LightningDataModule):

  def __init__(self, args,data = None):
    super().__init__()
   

  def setup(self, stage=None):
    
    print('preparing data')
    self.wiki = Dataset_processing()
    self.sum = SumDataset()
    self.train_dataset = torch.utils.data.ConcatDataset([self.sum, self.wiki])
    
    args.n_training_steps = len(self.train_dataset)//args.train_batch_size
    args.n_warmup_steps = args.n_training_steps//10

   
  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=args.train_batch_size,
      shuffle=True
    )

  '''def val_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=args.eval_batch_size,
      num_workers=2
    )

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.eval_batch_size,
      num_workers=2
    )'''

In [ ]:
#wiki = Dataset_processing()
#sum = SumDataset()
#train_dataset = torch.utils.data.ConcatDataset([sum, wiki])
    

KeyboardInterrupt: ignored

In [ ]:
train_dataloader = DataLoader(
      train_dataset,
      batch_size=args.train_batch_size,
      shuffle=True
    )

In [ ]:
from x_transformers import TransformerWrapper, Decoder
from x_transformers.autoregressive_wrapper import AutoregressiveWrapper

gpt = AutoregressiveWrapper(
      TransformerWrapper(
        num_tokens = 50259,
        max_seq_len = 1024,
        attn_layers = Decoder(
            dim = 512,
            depth = 12,
            heads = 8,
            #ff_swish = True, # set this to True
           # ff_glu = True  
               
        )
    ))

In [ ]:
#model.zero_grad()

In [ ]:
#len(train_dataloader)

In [ ]:
from tqdm import tqdm
LEARNING_RATE = 1e-4

def train_loop():
  optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

  GRADIENT_ACCUMULATE_EVERY = 4
  NUM_BATCHES = 10000000
  epoch_iterator = tqdm(train_dataloader, desc="Iteration")

  for step, inputs in enumerate(epoch_iterator, start=1):

      gpt.train()


    
      loss = gpt(inputs)
      loss.backward()

      #print(f'training loss: {loss.item()}')
      torch.nn.utils.clip_grad_norm_(gpt.parameters(), 0.5)
      optim.step()
      optim.zero_grad()
          
      gpt.zero_grad()
      

In [ ]:
#data_wiki[0]

NameError: ignored

# torch light

In [ ]:
def test_forward(data_wiki):
  loss = gpt.forward(data_wiki[0].unsqueeze(0))

In [ ]:

from x_transformers import TransformerWrapper, Decoder
from x_transformers.autoregressive_wrapper import AutoregressiveWrapper

class DecoderTrainer(pl.LightningModule):

  def __init__(self, args):
    super().__init__()
    
    self.gpt = AutoregressiveWrapper(TransformerWrapper(
        num_tokens = 50259,
        max_seq_len = 1024,
        attn_layers = Decoder(
            dim = 512,
            depth = 12,
            heads = 8,
            
              
        )
    ))
    if not args.n_training_steps:
      self.n_training_steps = 25000#args.n_training_steps
    else:
      self.n_training_steps = args.n_training_steps
      self.n_warmup_steps = 10
    

  def forward(self, input_ids):
    
    print(input_ids)
    loss = self.gpt.forward(input_ids)
    return loss

  def training_step(self, batch, batch_idx):
    
    loss = self.gpt(batch)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss}

  def validation_step(self, batch, batch_idx):
  
    loss = self.gpt(batch)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    
    
    loss = self.gpt(batch)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def training_epoch_end(self, outputs):
    
    pass

  def configure_optimizers(self):

    optimizer = torch.optim.Adam(self.parameters(), lr=args.lr)

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
      optimizer, 
      max_lr=args.lr,
      final_div_factor=500, 
      steps_per_epoch=args.n_training_steps,
      epochs=10
    )

    

    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )

In [ ]:
data_module = DecoderDataModule(args)

In [ ]:
model = DecoderTrainer(args=args)

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

early_stopping_callback = EarlyStopping(monitor='train_loss', patience=2)

In [ ]:
class CheckpointEveryNSteps(pl.Callback):
    """
    Save a checkpoint every N steps, instead of Lightning's default that checkpoints
    based on validation loss.
    """

    def __init__(
        self,
        save_step_frequency,
        prefix="N-Step-Checkpoint",
        use_modelcheckpoint_filename=False,
    ):
        """
        Args:
            save_step_frequency: how often to save in steps
            prefix: add a prefix to the name, only used if
                use_modelcheckpoint_filename=False
            use_modelcheckpoint_filename: just use the ModelCheckpoint callback's
                default filename, don't use ours.
        """
        self.save_step_frequency = save_step_frequency
        self.prefix = prefix
        self.use_modelcheckpoint_filename = use_modelcheckpoint_filename

    def on_batch_end(self, trainer: pl.Trainer, _):
        """ Check if we should save a checkpoint after every train batch """
        epoch = trainer.current_epoch
        global_step = trainer.global_step
        if global_step % self.save_step_frequency == 0:
            if self.use_modelcheckpoint_filename:
                filename = trainer.checkpoint_callback.filename
            else:
                filename =  f"{self.prefix}_{epoch}_{global_step}.ckpt"
            ckpt_path = os.path.join(trainer.checkpoint_callback.dirpath, filename)
            trainer.save_checkpoint(ckpt_path)




In [ ]:
from pytorch_lightning.loggers import WandbLogger

wandb_logger = WandbLogger(project="Decoder")

wandb: Currently logged in as: alexwortega. Use `wandb login --relogin` to force relogin


In [ ]:
trainer = pl.Trainer(
  logger=wandb_logger,
  checkpoint_callback=[CheckpointEveryNSteps(save_step_frequency=20000)],
  callbacks=[early_stopping_callback],
  max_epochs=10,
  gpus=1,
  gradient_clip_val=0.5,
  accumulate_grad_batches=7,
  progress_bar_refresh_rate=30
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=[<__main__.CheckpointEveryNSteps object at 0x7f7b1ba84f90>])` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=[<__main__.CheckpointEveryNSteps object at 0x7f7b1ba84f90>])`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={p

In [ ]:
trainer.fit(model, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")


preparing data


Tokenizing: 100%|██████████| 100000/100000 [03:39<00:00, 454.86it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type                  | Params
-----------------------------------------------
0 | gpt  | AutoregressiveWrapper | 89.9 M
-----------------------------------------------
89.9 M    Trainable params
0         Non-trainable params
89.9 M    Total params
359.404   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

In [ ]:
point = '/content/Decoder/3bhxp6wf/checkpoints/epoch=0-step=129608.ckpt'
_fix_pl(point)

In [ ]:
torch.load('/content/fixed.pt')

In [1]:
from x_transformers import TransformerWrapper, Decoder
from x_transformers.autoregressive_wrapper import AutoregressiveWrapper
import torch
from transformers import  GPT2TokenizerFast

BASE_MODEL = "sberbank-ai/rugpt3small_based_on_gpt2"

tokenizer = GPT2TokenizerFast.from_pretrained(BASE_MODEL, bos_token='<|endoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')




def _fix_pl(path):
  d = torch.load(path)["state_dict"]
  checkpoint = {}
  for key in d.keys():
    checkpoint[key.replace('gpt.','')] = d[key]
  torch.save(checkpoint,'fixed.pt')


_fix_pl('Decoder/3uu9967k/checkpoints/epoch=8-step=26793.ckpt')



/home/alex/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
gpt = AutoregressiveWrapper(TransformerWrapper(
        num_tokens = 50259,
        max_seq_len = 1024,
        attn_layers = Decoder(
            dim = 512,
            depth = 12,
            heads = 8,
            rotary_pos_emb = True ,
            ff_swish = True, # set this to True
            ff_glu = True   
        )
    )).half().cuda()

gpt.load_state_dict(torch.load('fixed.pt'))



<All keys matched successfully>

In [24]:
enc = tokenizer.encode('Пушкин ', return_tensors='pt').cuda()
gen = gpt.generate(enc, 20)
print(tokenizer.decode(gen[0]))

14 ноября 1974 года получил степень бакалавра по вопросам посещения Божией Матери в Тельвалесарях. Л


In [17]:
enc = tokenizer.encode('Пушкин это', return_tensors='pt')

In [ ]:
enc

tensor([[  672, 14173,   481]])

RuntimeError: ignored

In [ ]:
a=0
while True:
  a+=1
  a-=1